<a href="https://colab.research.google.com/github/cykadelix/Library-project/blob/master/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from PIL import Image
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator


In [2]:
!nvidia-smi

Tue Nov 28 06:51:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    25W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
train_folder = '/content/drive/MyDrive/chest_xray/train'
test_folder = '/content/drive/MyDrive/chest_xray/test'
validation_folder = '/content/drive/MyDrive/chest_xray/val'

# Data augmentation for the training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Data augmentation for the validation set
validation_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Data augmentation for the test set (CHANGE FOR FUTURE TESTS)
test_datagen = ImageDataGenerator(
    rescale=1./255,
)

# Define batch size and image dimensions
batch_size = 32
img_height, img_width = 224, 224

# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_folder,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_folder,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_folder,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [4]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(512, activation='relu'),
     # Add a Dropout layer with a specified dropout rate (e.g., 0.5)
    Dropout(0.5),
    BatchNormalization(),

    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

In [7]:
# Custom loss function
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_function(y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1.0 - epsilon)

        # Compute focal loss
        loss = - alpha * (1 - y_pred)**gamma * y_true * tf.math.log(y_pred) \
               - (1 - alpha) * y_pred**gamma * (1 - y_true) * tf.math.log(1 - y_pred)

        return tf.reduce_mean(loss)

    return focal_loss_function

In [8]:
custom_optimizer = keras.optimizers.Adam(learning_rate=0.01)

model.compile(
    loss=focal_loss(),
    optimizer='adam',
    metrics=['accuracy']
)

In [9]:
epochs = 10
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator), # batch_size,
    validation_data=validation_generator,
    validation_steps=len(validation_generator), # batch_size,
    epochs=epochs
)

Epoch 1/10
163/163 [==============================] - 103s 615ms/step - loss: 0.0319 - accuracy: 0.8932 - val_loss: 0.1876 - val_accuracy: 0.6875
Epoch 2/10
163/163 [==============================] - 100s 614ms/step - loss: 0.0259 - accuracy: 0.9011 - val_loss: 0.1176 - val_accuracy: 0.6250
Epoch 3/10
163/163 [==============================] - 100s 615ms/step - loss: 0.0257 - accuracy: 0.9013 - val_loss: 0.0589 - val_accuracy: 0.7500
Epoch 4/10
163/163 [==============================] - 97s 592ms/step - loss: 0.0272 - accuracy: 0.8930 - val_loss: 0.8700 - val_accuracy: 0.5000
Epoch 5/10
163/163 [==============================] - 96s 590ms/step - loss: 0.0265 - accuracy: 0.8934 - val_loss: 0.4770 - val_accuracy: 0.5000
Epoch 6/10
163/163 [==============================] - 96s 589ms/step - loss: 0.0242 - accuracy: 0.9055 - val_loss: 0.1542 - val_accuracy: 0.6875
Epoch 7/10
163/163 [==============================] - 95s 585ms/step - loss: 0.0268 - accuracy: 0.8938 - val_loss: 0.2954 - val

In [12]:
test_loss, test_accuracy = model.evaluate(test_generator, verbose=2)
print(f"Test accuracy: {test_accuracy}")


20/20 - 5s - loss: 0.0538 - accuracy: 0.8942 - 5s/epoch - 263ms/step
Test accuracy: 0.8942307829856873
